# CRISPRi for transcription regulation

First import the API and other packages we need:

In [ ]:
from flapjack_api import FlapjackSession, layout_print, load_figures
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as io
import json
import getpass
from scipy.optimize import curve_fit
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

Next we connect to Flapjack by creating a session with our login:

In [ ]:
user = 'tim'
pwd = getpass.getpass()
session = FlapjackSession('http://localhost:8989', user, pwd)

In [ ]:
filters = {
          'study':['CRISPRi repression']
          }
expr_dnas, _ = session.get_dnas(filters)
dnas = list(np.unique(expr_dnas['names']))
dnas = [d for d in dnas if d not in (['None'], ['CcaT', 'pBM3R1-B2_cyan'],  ['CcaT', 'TMA3'],  ['CcaT', 'StdStd'])]
dnas

## Plot an induction curve using mean expression

In [ ]:
session = FlapjackSession('http://localhost:8989', user, pwd)
filter = { 
          'study': ['CRISPRi repression'],
          'assay': ['RV7 - python', 'RV8 - python', 'RV9 - python'],
          'groupby1': 'sample__assay__study__name',
          'groupby2': 'name',
          'groupby3': 'sample__dna__names',
          'dna': dnas,
          'measure': 'mean_expression',
          'density_name': 'OD',
          'inducer': ['IPTG']
         }
figs,s = session.plot_induction_curve(filter)

In [ ]:
fig = figs['CRISPRi repression']
# Layout the figure better for our purposes
layout_print(fig, width=4.5, aspect=3.25, font_size=6)

fig['layout']['annotations'][0]['text'] = 'Turquoise'
fig['layout']['annotations'][1]['text'] = 'OD'
fig['layout']['annotations'][2]['text'] = 'Venus'
fig.update_yaxes(rangemode='normal')
#fig.update_yaxes(range=[0,8e4], row=1, col=1)
fig.show()

io.write_image(fig, '../Doc/CRISPR_induction_curves.png')


## Generate a kymograph of gene expression rate at different inducer concentrations

In the Flapjack web app select... blah bla... Save the figure as JSON by clicking the button.

Now we can load the figure into this notebook by specifying the file location below. We also reformat the figure for pubication requirements and then save it as a PNG at 300 dpi:

In [ ]:
figs = load_figures('../Doc/TMA5_kymograph.json')
name_map = {
    'CFP': 'Turquoise',
    'YFP': 'Venus',
    'OD': 'OD'
}
for name, fig in figs.items():
    fig = layout_print(fig, width=6.5/3, aspect=1.5, font_size=6.)
    fig.update_traces(colorscale='portland')
    fig['layout']['annotations'][0]['text'] = name_map[name]
    fig.show()
    io.write_image(fig, '../Doc/TMA5_kymo_'+name+'.png')

## Analysing induction curves with Flapjack

We can get the data of the induction curve, the concentrations of inducer and their corresponding expression level for each reporter, directly from the API. The API also returns us the fitted Hill function parameters.

In [ ]:
session = FlapjackSession('http://localhost:8989', user, pwd)
filter = { 
          'study': ['CRISPRi repression'],
            'assay': ['RV7 - python', 'RV8 - python', 'RV9 - python'],
          'dna': dnas,
          'measure': 'mean_expression',
          'density_name': 'OD',
          'inducer': ['IPTG']
         }
induction_params = pd.DataFrame()
induction_curves = {}
for dna in dnas:
    filter['dna'] = [dna]
    dna_str = '+'.join(dna)
    curve,params,s = session.get_induction_curve(filter)
    params['dna'] = [dna_str]*3
    induction_curves[dna_str] = curve
    induction_params = induction_params.append(params)
    

Here we plot the parameters for each DNA (plasmid) that we got above. We can see that all are severely altered by induction of dCas9.

In [ ]:
induction_params = induction_params.sort_values('name')

print(induction_params.head())

# Map the signal names to the protein names
name_map = {
    'YFP': 'Venus',
    'CFP': 'Turquoise',
    'OD': 'OD'
}

subplot_titles = ['a', 'b', 'k', 'n'] + [''] * 8
fig = make_subplots(rows=3, 
                    cols=4, 
                    vertical_spacing=0.1, 
                    horizontal_spacing=0.1,
                    shared_xaxes=True,
                    subplot_titles=subplot_titles
                   )
col = 1
for p in ['a', 'b', 'k', 'n']:
    row = 1
    for name,data in induction_params.groupby('name'):
        error_y = dict(type='data', # value of error bar given in data coordinates
                        array=data[p+' std'],
                        visible=True)
        marker = dict(color='rgb(128,128,128)')
        bar = go.Bar(x=data['dna'], 
                     y=data[p],
                     error_y=error_y, 
                     name=p, 
                     legendgroup=p,
                     marker=marker,
                     showlegend=False
                    )
        fig.add_trace(bar, row=row, col=col)
        if col==1:
            fig.update_yaxes(title_text=name_map[name], row=row, col=col)
        row += 1
    col += 1

fig = layout_print(fig, width=4.5, aspect=1.5, font_size=6.)
fig.show()
    

## Plot the induction curves from the returned data

In [ ]:
fig,axs = plt.subplots(1,3, figsize=(12,3))
fig.suptitle(dna)
for dna,curve in induction_curves.items():
    i = 0
    for name,data in curve.groupby('name'):
        ax = axs[i]
        ax.plot(np.log10(data['Concentration']), data['Expression'], '.')
        ax.set_title(name)
        i += 1
    plt.tight_layout()
plt.legend(induction_curves.keys())